In [17]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import pickle
from tqdm import tqdm
out_put_formatter = '../../for_shiyi/{}_oof.pkl'

In [18]:
nn_report = pd.read_csv('../../data/nn/report.csv')
train_instance = pd.read_pickle(FILE.train_ori.value)[['instance_id']].copy()
test_instance = pd.read_pickle(FILE.test_ori.value)[['instance_id']].copy()
train_index = pickle.load(open(FILE.train_index.value,'rb'))
holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))

In [23]:
oof = train_instance.loc[train_index].copy()
holdout = train_instance.loc[holdout_index].copy()
test = test_instance.copy()

# load nn

In [24]:
filter_col = 'cv_mean'
thred = 0.42
nn_available_file = nn_report[nn_report[filter_col]<thred].fileName
print(len(nn_available_file))

for each_file in tqdm(nn_available_file):
    train_load = pd.read_pickle('../../data/nn/train/{}'.format(each_file)).rename({'predicted_score':each_file},axis=1)
    test_load = pd.read_pickle('../../data/nn/test/{}'.format(each_file)).rename({'predicted_score':each_file},axis=1)
    oof = oof.merge(train_load,how='inner',on='instance_id')
    holdout = holdout.merge(train_load,how='inner',on='instance_id')
    test = test.merge(test_load,how='inner',on='instance_id')

  0%|          | 0/30 [00:00<?, ?it/s]

30


100%|██████████| 30/30 [00:17<00:00,  1.73it/s]


In [26]:
print(oof.shape)
print(holdout.shape)
print(test.shape)

(961626, 31)
(40024, 31)
(40024, 31)


# save

In [27]:
oof.to_pickle(out_put_formatter.format('train'))
holdout.to_pickle(out_put_formatter.format('holdout'))
test.to_pickle(out_put_formatter.format('test'))